# Final Clean Hybrid MuReD Notebook v3

DenseNet121 + ResNet101 hybrid, image size 192, FOV extraction, blur filtering (drop bottom 10%), pos_weight, WeightedRandomSampler, Albumentations, MixUp, gradual unfreeze, AMP, checkpointing & resume, threshold tuning, MC-Dropout uncertainty, Grad-CAM, failure gallery, full evaluation metrics (AUC/AP/F1) — ready to run on RTX 3060/3070/4090.

**Paths** used:
- Images: `/mnt/data/images/`
- Train CSV: `/mnt/data/train_data.csv`
- Test CSV: `/mnt/data/test_data.csv`

Run cells top → bottom. Adjust paths if needed.

In [1]:
# === Imports & Config ===
import os, time, math, random, gc
from glob import glob
from pathlib import Path
import numpy as np, pandas as pd
from PIL import Image
import cv2
import json

import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from torch.cuda.amp import autocast, GradScaler

import torchvision.models as models
import albumentations as A
from albumentations.pytorch import ToTensorV2

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, average_precision_score, precision_recall_curve, f1_score, precision_score, recall_score

# Reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Paths & settings
# DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DEVICE = torch.device('cpu')
IMG_DIR = './MuReD_dataset/images'
TRAIN_CSV = './MuReD_dataset/train_data.csv'
TEST_CSV = './MuReD_dataset/test_data.csv'
CACHE_DIR = "./cache/new"
CACHE_FILE = os.path.join(CACHE_DIR, "blur_cache.pkl")

IMG_SIZE = 192
BATCH_SIZE = 32
NUM_WORKERS = 0

# Training schedule for RTX GPUs
WARMUP_EPOCHS = 5
MAIN_EPOCHS = 30
TOTAL_EPOCHS = WARMUP_EPOCHS + MAIN_EPOCHS

CKPT_DIR = './MuReD_dataset/checkpoints/'
os.makedirs(CKPT_DIR, exist_ok=True)
OUT_DIR = './MuReD_dataset/output/'
os.makedirs(OUT_DIR, exist_ok=True)

print('Device:', DEVICE, 'IMG_DIR exists:', os.path.exists(IMG_DIR))

Device: cpu IMG_DIR exists: True


In [2]:
# === FOV extraction and blur metric helpers ===
import numpy as np
import cv2

def extract_fov(img_rgb, tol=7):
    """Crop image to field-of-view (non-black area). img_rgb: HxWx3 uint8."""
    if img_rgb is None:
        return img_rgb
    gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)
    mask = gray > tol
    if mask.sum() == 0:
        return img_rgb
    coords = np.argwhere(mask)
    y0, x0 = coords.min(axis=0)
    y1, x1 = coords.max(axis=0)
    crop = img_rgb[y0:y1+1, x0:x1+1]
    return crop

def blur_score(img_rgb):
    """Laplacian variance as blur score."""
    if img_rgb is None:
        return 0.0
    gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)
    return float(cv2.Laplacian(gray, cv2.CV_64F).var())

print('FOV and blur helpers ready')

FOV and blur helpers ready


In [3]:
# === Blur Score Cache System (Optimized) ===

os.makedirs(CACHE_DIR, exist_ok=True)

# Load train/test CSV
train_df = pd.read_csv(TRAIN_CSV)
test_df = pd.read_csv(TEST_CSV)
print('Train shape:', train_df.shape, 'Test shape:', test_df.shape)

if 'ID' not in train_df.columns:
    raise ValueError("train_data.csv must contain ID column")

label_cols = [c for c in train_df.columns if c != 'ID']
print(f"Detected label columns ({len(label_cols)}):", label_cols)

# === Load from cache if available ===
if os.path.exists(CACHE_FILE):
    cached = pd.read_pickle(CACHE_FILE)
    train_df['blur_score'] = cached['blur_score']
    print("✔ Loaded blur scores from cache:", CACHE_FILE)

else:
    print("✖ Cache missing — computing blur scores...")

    blur_scores = []
    for idx, row in train_df.iterrows():
        img_id = str(row['ID'])
        p = None

        # Try common extensions
        for ext in ['.jpg','.jpeg','.png','.tif','.tiff']:
            cand = os.path.join(IMG_DIR, img_id + ext)
            if os.path.exists(cand):
                p = cand
                break

        # Direct match
        if p is None:
            cand2 = os.path.join(IMG_DIR, img_id)
            if os.path.exists(cand2):
                p = cand2

        # Fallback: any file starting with ID
        if p is None:
            matches = glob(os.path.join(IMG_DIR, img_id + '*'))
            p = matches[0] if matches else None

        if p is None:
            blur_scores.append(np.nan)
            continue

        try:
            img = np.array(Image.open(p).convert("RGB"))
            fov = extract_fov(img)
            b = blur_score(fov)
            blur_scores.append(b)
        except:
            blur_scores.append(np.nan)

    train_df['blur_score'] = blur_scores

    # Save to cache
    train_df[['blur_score']].to_pickle(CACHE_FILE)
    print("✔ Cached blur scores to:", CACHE_FILE)

# === Drop bottom 10% low sharpness ===
n_drop = int(0.10 * len(train_df))
train_df = train_df.sort_values('blur_score', na_position='first').iloc[n_drop:].reset_index(drop=True)
print("After dropping bottom 10%:", len(train_df))

# Labels
labels_all = train_df[label_cols].values.astype(np.float32)
print("Labels shape:", labels_all.shape)

Train shape: (1764, 21) Test shape: (444, 21)
Detected label columns (20): ['DR', 'NORMAL', 'MH', 'ODC', 'TSLN', 'ARMD', 'DN', 'MYA', 'BRVO', 'ODP', 'CRVO', 'CNV', 'RS', 'ODE', 'LS', 'CSR', 'HTR', 'ASR', 'CRS', 'OTHER']
✔ Loaded blur scores from cache: ./cache/new\blur_cache.pkl
After dropping bottom 10%: 1588
Labels shape: (1588, 20)


In [4]:
# === Train/Val split (iterative stratify if available) ===
try:
    from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit
    splitter = MultilabelStratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=SEED)
    train_idx, val_idx = next(splitter.split(train_df, labels_all))
    print('Used MultilabelStratifiedShuffleSplit')
except Exception as e:
    print('iterative stratify not available, falling back to simple split:', e)
    train_idx, val_idx = train_test_split(np.arange(len(train_df)), test_size=0.2, random_state=SEED)

df_tr = train_df.iloc[train_idx].reset_index(drop=True)
df_val = train_df.iloc[val_idx].reset_index(drop=True)
y_train = df_tr[label_cols].values.astype(np.float32)
y_val = df_val[label_cols].values.astype(np.float32)
print('Train/Val sizes:', df_tr.shape, df_val.shape)

iterative stratify not available, falling back to simple split: No module named 'iterstrat'
Train/Val sizes: (1270, 22) (318, 22)


In [5]:
# === Compute pos_weight for BCEWithLogitsLoss ===
pos = y_train.sum(axis=0).astype(np.float32)
neg = y_train.shape[0] - pos
pos_weight = np.clip((neg / (pos + 1e-6)).astype(np.float32), 1.0, 50.0)
pos_weight_tensor = torch.tensor(pos_weight).to(DEVICE)
num_classes = y_train.shape[1]
print('pos_weight (first 10):', pos_weight[:10])

pos_weight (first 10): [ 3.6014493  3.4561403 14.119047   7.3006535 12.804348  12.368421
 12.092784  34.27778   27.863636  33.324326 ]


In [6]:
# === Augmentations (Albumentations) ===
train_tfms = A.Compose([
    A.Resize(IMG_SIZE, IMG_SIZE),

    A.RandomResizedCrop(size=(IMG_SIZE, IMG_SIZE), scale=(0.8, 1.0), p=0.6),

    A.HorizontalFlip(p=0.5),
    A.Rotate(limit=20, p=0.3),
    A.CLAHE(p=0.5),
    A.RandomBrightnessContrast(p=0.5),

    A.OneOf([
        A.CoarseDropout(max_holes=8, max_height=16, max_width=16),
        A.CoarseDropout(max_holes=8, min_holes=8, max_height=16, max_width=16),
    ], p=0.3),

    A.Normalize(),
    ToTensorV2(),
])

val_tfms = A.Compose([
    A.Resize(IMG_SIZE, IMG_SIZE),
    A.Normalize(),
    ToTensorV2()
])

print(f'Transforms set (IMG_SIZE={IMG_SIZE})')


Transforms set (IMG_SIZE=192)


C:\Users\anves\AppData\Local\Temp\ipykernel_16012\33596865.py:13: UserWarning: Argument(s) 'max_holes, max_height, max_width' are not valid for transform CoarseDropout
  A.CoarseDropout(max_holes=8, max_height=16, max_width=16),
C:\Users\anves\AppData\Local\Temp\ipykernel_16012\33596865.py:14: UserWarning: Argument(s) 'max_holes, min_holes, max_height, max_width' are not valid for transform CoarseDropout
  A.CoarseDropout(max_holes=8, min_holes=8, max_height=16, max_width=16),


In [7]:
# === MuReD Dataset class ===
class MuReDDataset(Dataset):
    def __init__(self, df, label_cols, transform, img_dir=IMG_DIR):
        self.df = df.reset_index(drop=True)
        self.label_cols = label_cols
        self.transform = transform
        self.img_dir = img_dir

    def _find_image(self, img_id):
        for ext in ['.jpg','.jpeg','.png','.tif','.tiff']:
            p = os.path.join(self.img_dir, str(img_id) + ext)
            if os.path.exists(p):
                return p
        p2 = os.path.join(self.img_dir, str(img_id))
        if os.path.exists(p2):
            return p2
        matches = glob(os.path.join(self.img_dir, str(img_id) + '*'))
        if matches:
            return matches[0]
        return None

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_id = row['ID']
        p = self._find_image(img_id)
        if p is None:
            img_np = np.zeros((IMG_SIZE, IMG_SIZE, 3), dtype=np.uint8)
        else:
            img = Image.open(p).convert('RGB')
            img_np = np.array(img)
            # apply FOV extraction if available
            try:
                img_np = extract_fov(img_np)
            except Exception:
                pass
        augmented = self.transform(image=img_np)
        img_t = augmented['image']
        labels = torch.tensor(row[self.label_cols].values.astype(np.float32))
        return img_t, labels, str(img_id)

In [8]:
# === DataLoaders & WeightedRandomSampler ===
label_freq = y_train.sum(axis=0) + 1e-6
sample_weights = []
for lbl in y_train:
    present = lbl.astype(bool)
    if present.sum() == 0:
        w = 1.0
    else:
        w = (1.0 / label_freq[present]).sum()
    sample_weights.append(w)

sampler = WeightedRandomSampler(sample_weights, num_samples=len(sample_weights), replacement=True)

train_ds = MuReDDataset(df_tr, label_cols, train_tfms, img_dir=IMG_DIR)
val_ds = MuReDDataset(df_val, label_cols, val_tfms, img_dir=IMG_DIR)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, sampler=sampler, num_workers=NUM_WORKERS, pin_memory=True)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)

print('DataLoaders ready. Train batches:', len(train_loader), 'Val batches:', len(val_loader))

DataLoaders ready. Train batches: 40 Val batches: 10


In [9]:
# === Hybrid model: DenseNet121 + ResNet101 (concatenation fusion) ===
class HybridNet(nn.Module):
    def __init__(self, num_classes, dropout=0.3, pretrained=True):
        super().__init__()
        densenet = models.densenet121(pretrained=pretrained)
        self.dense_features = densenet.features  # out channels 1024
        resnet = models.resnet101(pretrained=pretrained)
        self.resnet_backbone = nn.Sequential(*list(resnet.children())[:-2])  # out channels 2048
        self.pool = nn.AdaptiveAvgPool2d((1,1))
        fused_size = 1024 + 2048
        self.classifier = nn.Sequential(
            nn.Linear(fused_size, 1024),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(1024, num_classes)
        )

    def forward(self, x):
        d = self.dense_features(x)  # B x 1024 x H x W
        d = self.pool(d).view(d.size(0), -1)
        r = self.resnet_backbone(x)  # B x 2048 x H' x W'
        r = self.pool(r).view(r.size(0), -1)
        fused = torch.cat([d, r], dim=1)
        out = self.classifier(fused)
        return out

model = HybridNet(num_classes=num_classes, pretrained=True).to(DEVICE)
print('Model created, parameters:', sum(p.numel() for p in model.parameters()))

c:\Users\anves\OneDrive\Desktop\Siddhi Jaiswal Model\venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\anves\OneDrive\Desktop\Siddhi Jaiswal Model\venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
c:\Users\anves\OneDrive\Desktop\Siddhi Jaiswal Model\venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Re

Model created, parameters: 52621268


In [10]:
# === Loss, optimizer (head-only warmup), scheduler placeholders ===
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight_tensor)
# Freeze all except classifier
for name, p in model.named_parameters():
    if 'classifier' in name:
        p.requires_grad = True
    else:
        p.requires_grad = False

optimizer = optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-3, weight_decay=1e-4)
# scaler for AMP
scaler = GradScaler()
print('Criterion, optimizer, AMP scaler ready')

Criterion, optimizer, AMP scaler ready


C:\Users\anves\AppData\Local\Temp\ipykernel_16012\1991175779.py:12: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


In [11]:
# === MixUp and training/evaluation functions ===
import numpy as np

def mixup_data(x, y, alpha=0.4):
    if alpha <= 0:
        return x, y, 1.0
    lam = np.random.beta(alpha, alpha)
    idx = torch.randperm(x.size(0)).to(x.device)
    mixed_x = lam * x + (1 - lam) * x[idx]
    mixed_y = lam * y + (1 - lam) * y[idx]
    return mixed_x, mixed_y, lam

def train_one_epoch(model, loader, optimizer, criterion, scaler, mixup_alpha=0.4):
    model.train()
    running_loss = 0.0
    n = 0

    print("→ train_one_epoch: starting epoch, waiting for first batch...")

    for batch_idx, (imgs, labels, _) in enumerate(loader):

        # DEBUG: dataloader health
        if batch_idx % 10 == 0:
            print(f"  Batch {batch_idx}/{len(loader)} loaded")

        imgs = imgs.to(DEVICE)
        labels = labels.to(DEVICE)

        imgs, labels, _ = mixup_data(imgs, labels, alpha=mixup_alpha)

        optimizer.zero_grad()

        # Forward pass
        with torch.amp.autocast("cuda"):
            logits = model(imgs)
            loss = criterion(logits, labels)

        print(f"    Forward OK (batch {batch_idx})")

        # Backward + step
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        print(f"    Backward+Step OK (batch {batch_idx})")

        running_loss += float(loss.item()) * imgs.size(0)
        n += imgs.size(0)

    return running_loss / (n + 1e-12)

def predict_loader(model, loader):
    model.eval()
    preds = []; targs = []; names = []
    with torch.no_grad():
        for imgs, labels, img_ids in loader:
            imgs = imgs.to(DEVICE)
            out = torch.sigmoid(model(imgs))
            preds.append(out.cpu().numpy())
            targs.append(labels.numpy())
            names += img_ids
    if len(preds)==0:
        return np.zeros((0, num_classes)), np.zeros((0, num_classes)), []
    return np.vstack(preds), np.vstack(targs), names

print('Training/eval functions ready')

Training/eval functions ready


In [20]:
# === Checkpoint save/load utilities ===
def save_checkpoint(state, filename='checkpoint.pth'):
    path = os.path.join(CKPT_DIR, filename)
    torch.save(state, path)
    print('Saved checkpoint:', path)

def load_checkpoint(model, optimizer=None, scaler=None, filename='checkpoint.pth'):
    path = os.path.join(CKPT_DIR, filename)
    if not os.path.exists(path):
        print(f'No checkpoint found at {path}')
        return None

    ck = torch.load(path, map_location=DEVICE)
    model.load_state_dict(ck['model_state_dict'])
    
    # ========== Safe Optimizer Load ==========
    if optimizer is not None and 'optimizer_state_dict' in ck:
        try:
            optimizer.load_state_dict(ck['optimizer_state_dict'])
            print("✓ Optimizer state loaded successfully")
        except ValueError as e:
            print("⚠️ Optimizer state NOT loaded (param group mismatch). Skipping.")
            print("Reason:", str(e))
            print("→ Continuing with fresh optimizer.")
    
    # ========== Safe Scaler Load ==========
    if scaler is not None and 'scaler_state_dict' in ck:
        try:
            scaler.load_state_dict(ck['scaler_state_dict'])
            print("✓ Scaler loaded successfully")
        except Exception as e:
            print("⚠️ Scaler NOT loaded. Skipping.")
            print("Reason:", str(e))

    print('Loaded checkpoint | epoch:', ck.get('epoch', '?'),
          '| best_map:', ck.get('best_map', '?'))

    return ck

In [21]:
# === Gradual unfreeze: unfreeze denseblock4 & resnet layer4 ===
# Unfreeze classifier + last blocks
for name, p in model.named_parameters():
    if 'classifier' in name:
        p.requires_grad = True
    if 'denseblock4' in name or 'transition3' in name:
        p.requires_grad = True
    if 'resnet_backbone.7' in name or 'resnet_backbone.6' in name or 'resnet_backbone.8' in name or 'resnet_backbone.5' in name:
        p.requires_grad = True

backbone_params = [p for n,p in model.named_parameters() if p.requires_grad and ('classifier' not in n)]
head_params     = [p for n,p in model.named_parameters() if p.requires_grad and ('classifier' in n)]

param_groups = [
    {'params': backbone_params, 'lr': 1e-5},
    {'params': head_params,     'lr': 1e-4},
]

optimizer = optim.AdamW(param_groups, weight_decay=1e-4)
print("Optimizer reset with discriminative LR")

Optimizer reset with discriminative LR


In [22]:
# === Warmup: head-only training ===
start_epoch = 0
best_map = 0.0

# Load ONLY model + scaler, NOT optimizer
ck = load_checkpoint(model, optimizer=None, scaler=scaler, filename='main_checkpoint.pth')

if ck is not None:
    start_epoch = ck.get('epoch', 0)
    best_map = ck.get('best_map', 0.0)
    print('Resuming from checkpoint, start_epoch:', start_epoch)

# Warmup epochs (train only classifier)
for e in range(start_epoch, WARMUP_EPOCHS):
    t0 = time.time()
    print(f"→ Starting Warmup Epoch {e+1}/{WARMUP_EPOCHS}")
    loss = train_one_epoch(model, train_loader, optimizer, criterion, scaler, mixup_alpha=0.0)
    dt = time.time() - t0
    print(f'Warmup Epoch {e+1}/{WARMUP_EPOCHS} - loss: {loss:.4f} - time: {dt:.1f}s')
    # save checkpoint
    save_checkpoint({
        'epoch': e+1,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scaler_state_dict': scaler.state_dict(),
        'best_map': best_map
    }, filename='main_checkpoint.pth')

✓ Scaler loaded successfully
Loaded checkpoint | epoch: 5 | best_map: 0.0
Resuming from checkpoint, start_epoch: 5


In [24]:
# === Full training (main) ===
from sklearn.exceptions import UndefinedMetricWarning
import warnings
warnings.filterwarnings('ignore', category=UndefinedMetricWarning)

E_start = WARMUP_EPOCHS
best_map = best_map if 'best_map' in globals() else 0.0

# Optionally use scheduler; here we use ReduceLROnPlateau on val mAP
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=3)

for epoch in range(E_start, E_start + MAIN_EPOCHS):
    t0 = time.time()
    train_loss = train_one_epoch(model, train_loader, optimizer, criterion, scaler, mixup_alpha=0.4)
    preds, targs, names = predict_loader(model, val_loader)
    if preds.shape[0] > 0:
        try:
            macro_auc = roc_auc_score(targs, preds, average='macro')
        except Exception:
            macro_auc = float('nan')
        macro_ap = average_precision_score(targs, preds, average='macro')
    else:
        macro_auc = macro_ap = 0.0
    dt = time.time() - t0
    print(f'Epoch {epoch+1}/{E_start+MAIN_EPOCHS} - train_loss: {train_loss:.4f} - val_macro_auc: {macro_auc:.4f} - val_macro_ap: {macro_ap:.4f} - time: {dt:.1f}s')

    old_lrs = [g['lr'] for g in optimizer.param_groups]
    # scheduler step
    scheduler.step(macro_ap)
    new_lrs = [g['lr'] for g in optimizer.param_groups]

    if new_lrs != old_lrs:
        print(f"LR reduced: {old_lrs} → {new_lrs}")

    # save checkpoint (main) and best
    ck_state = {
        'epoch': epoch+1,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scaler_state_dict': scaler.state_dict(),
        'best_map': best_map
    }
    save_checkpoint(ck_state, filename='main_checkpoint.pth')

    if macro_ap > best_map:
        best_map = macro_ap
        save_checkpoint(ck_state, filename='best_model.pth')
        print('New best model saved with mAP:', best_map)

→ train_one_epoch: starting epoch, waiting for first batch...
  Batch 0/40 loaded
    Forward OK (batch 0)
    Backward+Step OK (batch 0)
    Forward OK (batch 1)
    Backward+Step OK (batch 1)
    Forward OK (batch 2)
    Backward+Step OK (batch 2)
    Forward OK (batch 3)
    Backward+Step OK (batch 3)
    Forward OK (batch 4)
    Backward+Step OK (batch 4)
    Forward OK (batch 5)
    Backward+Step OK (batch 5)
    Forward OK (batch 6)
    Backward+Step OK (batch 6)
    Forward OK (batch 7)
    Backward+Step OK (batch 7)
    Forward OK (batch 8)
    Backward+Step OK (batch 8)
    Forward OK (batch 9)
    Backward+Step OK (batch 9)
  Batch 10/40 loaded
    Forward OK (batch 10)
    Backward+Step OK (batch 10)
    Forward OK (batch 11)
    Backward+Step OK (batch 11)
    Forward OK (batch 12)
    Backward+Step OK (batch 12)
    Forward OK (batch 13)
    Backward+Step OK (batch 13)
    Forward OK (batch 14)
    Backward+Step OK (batch 14)
    Forward OK (batch 15)
    Backward+Step OK 

In [26]:
# === Threshold tuning (per-class) and detailed metrics ===
preds, targs, names = predict_loader(model, val_loader)
best_thr = np.zeros(num_classes)
for i in range(num_classes):
    try:
        p, r, th = precision_recall_curve(targs[:,i], preds[:,i])
        f1 = 2*p*r/(p+r+1e-8)
        if len(f1) > 1:
            best_thr[i] = th[np.argmax(f1[:-1])]
        else:
            best_thr[i] = 0.5
    except Exception:
        best_thr[i] = 0.5
print('Best thresholds (first 10):', best_thr[:10])

# Binarize using thresholds and compute metrics
bin_preds = (preds >= best_thr.reshape(1,-1)).astype(int)
per_class_f1 = [f1_score(targs[:,i], bin_preds[:,i]) for i in range(num_classes)]
per_class_precision = [precision_score(targs[:,i], bin_preds[:,i], zero_division=0) for i in range(num_classes)]
per_class_recall = [recall_score(targs[:,i], bin_preds[:,i], zero_division=0) for i in range(num_classes)]

for c, f, p, r in zip(label_cols, per_class_f1, per_class_precision, per_class_recall):
    print(f"{c}: F1={f:.3f}, Prec={p:.3f}, Rec={r:.3f}")

print('\nMacro AUC:', roc_auc_score(targs, preds, average='macro'))
print('Macro AP:', average_precision_score(targs, preds, average='macro'))

Best thresholds (first 10): [0.45183086 0.37426716 0.81934994 0.83328742 0.89129949 0.84718233
 0.56688976 0.80646276 0.91708744 0.89069253]
DR: F1=0.667, Prec=0.603, Rec=0.746
NORMAL: F1=0.722, Prec=0.602, Rec=0.903
MH: F1=0.659, Prec=0.540, Rec=0.844
ODC: F1=0.519, Prec=0.538, Rec=0.500
TSLN: F1=0.500, Prec=0.450, Rec=0.562
ARMD: F1=0.605, Prec=0.591, Rec=0.619
DN: F1=0.263, Prec=0.175, Rec=0.526
MYA: F1=0.864, Prec=0.864, Rec=0.864
BRVO: F1=0.455, Prec=1.000, Rec=0.294
ODP: F1=0.400, Prec=0.286, Rec=0.667
CRVO: F1=0.857, Prec=0.750, Rec=1.000
CNV: F1=0.750, Prec=0.818, Rec=0.692
RS: F1=0.500, Prec=0.400, Rec=0.667
ODE: F1=0.778, Prec=0.875, Rec=0.700
LS: F1=0.421, Prec=0.400, Rec=0.444
CSR: F1=0.385, Prec=0.250, Rec=0.833
HTR: F1=0.267, Prec=0.200, Rec=0.400
ASR: F1=0.133, Prec=0.091, Rec=0.250
CRS: F1=0.857, Prec=1.000, Rec=0.750
OTHER: F1=0.312, Prec=0.222, Rec=0.526

Macro AUC: 0.9002446269088908
Macro AP: 0.4857430544110577


In [31]:
# === MC-Dropout uncertainty estimation & save report ===
def enable_dropout(m):
    for mod in m.modules():
        if isinstance(mod, nn.Dropout):
            mod.train()

T = 12
enable_dropout(model)
all_preds = []
for t in range(T):
    p, _, _ = predict_loader(model, val_loader)
    all_preds.append(p)
all_preds = np.stack(all_preds)  # T x n x C
pred_mean = all_preds.mean(axis=0)
pred_std = all_preds.std(axis=0)

probs = pred_mean
bin_preds = (probs >= best_thr.reshape(1,-1)).astype(int)

rows = []
for i, img_name in enumerate(names):
    rows.append({
        'image': img_name,
        'true': ','.join([str(int(x)) for x in targs[i]]),
        'pred_top3': ';'.join([f"{label_cols[j]}:{probs[i,j]:.3f}" for j in np.argsort(-probs[i])[:3]]),
        'uncertainty_mean': float(pred_std[i].mean()),
        'probs_json': json.dumps({label_cols[j]: float(probs[i,j]) for j in range(num_classes)}),
        'bin_pred': ','.join([label_cols[j] for j in np.where(bin_preds[i]==1)[0]])
    })

df_report = pd.DataFrame(rows)
report_path = os.path.join(OUT_DIR, 'val_prediction_report_with_uncertainty.csv')
df_report.to_csv(report_path, index=False)
print('Saved prediction report to', report_path)

Saved prediction report to ./MuReD_dataset/output/val_prediction_report_with_uncertainty.csv


In [29]:
# === Grad-CAM for Hybrid Model (combine DenseNet & ResNet cams) ===
try:
    from pytorch_grad_cam import GradCAM
    from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
    from pytorch_grad_cam.utils.image import show_cam_on_image
except Exception as e:
    print('grad-cam not installed. pip install grad-cam to enable visualization:', e)

# find target layers
dense_target = None
resnet_target = None
for n, m in model.named_modules():
    if 'denseblock4' in n and isinstance(m, nn.Conv2d):
        dense_target = m
    if 'resnet_backbone' in n and 'layer4' in n:
        resnet_target = m

print('Dense target:', dense_target, 'ResNet target module:', resnet_target)

OUT_CAM_DIR = os.path.join(OUT_DIR, 'gradcam_overlays'); os.makedirs(OUT_CAM_DIR, exist_ok=True)

val_iter = iter(val_loader)
for idx in range(min(12, len(val_loader))):
    imgs, labels_batch, id_batch = next(val_iter)
    img_tensor = imgs[0].to(DEVICE)
    name = id_batch[0]
    probs_img = torch.sigmoid(model(img_tensor.unsqueeze(0))).detach().cpu().numpy()[0]
    topk = np.argsort(-probs_img)[:3]
    for cls_idx in topk:
        try:
            cam_d = GradCAM(model=model, target_layers=[dense_target], use_cuda=torch.cuda.is_available())
            cam_r = GradCAM(model=model, target_layers=[resnet_target], use_cuda=torch.cuda.is_available())
            gcam_d = cam_d(input_tensor=img_tensor.unsqueeze(0), targets=[ClassifierOutputTarget(int(cls_idx))])[0]
            gcam_r = cam_r(input_tensor=img_tensor.unsqueeze(0), targets=[ClassifierOutputTarget(int(cls_idx))])[0]
            cam_avg = (gcam_d + gcam_r) / 2.0
            model_img_path = None
            for ext in ['.jpg','.jpeg','.png','.tif','.tiff','']:
                cand = os.path.join(IMG_DIR, name + ext)
                if os.path.exists(cand):
                    model_img_path = cand; break
            if model_img_path is None:
                matches = glob(os.path.join(IMG_DIR, name + '*'))
                if len(matches)>0:
                    model_img_path = matches[0]
            if model_img_path is None:
                continue
            orig = np.array(Image.open(model_img_path).convert('RGB').resize((IMG_SIZE, IMG_SIZE)))/255.0
            viz = show_cam_on_image(orig, cam_avg, use_rgb=True)
            outp = os.path.join(OUT_CAM_DIR, f"{name}_cls{cls_idx}.jpg")
            import cv2
            cv2.imwrite(outp, viz)
        except Exception as e:
            pass

print('Saved Grad-CAM overlays to', OUT_CAM_DIR)

grad-cam not installed. pip install grad-cam to enable visualization: No module named 'pytorch_grad_cam'
Dense target: Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False) ResNet target module: None


KeyboardInterrupt: 

In [30]:
# === Failure-case gallery (save False Negatives) ===
FAIL_DIR = os.path.join(OUT_DIR, 'failure_gallery'); os.makedirs(FAIL_DIR, exist_ok=True)
bin_preds = (probs >= best_thr.reshape(1,-1)).astype(int)
for cls_idx, cls_name in enumerate(label_cols):
    fn_idx = np.where((targs[:,cls_idx]==1) & (bin_preds[:,cls_idx]==0))[0]
    for i, ix in enumerate(fn_idx[:6]):
        img_name = names[ix]
        try:
            p = None
            for ext in ['.jpg','.jpeg','.png','.tif','.tiff','']:
                cand = os.path.join(IMG_DIR, img_name + ext)
                if os.path.exists(cand):
                    p = cand; break
            if p is None:
                p = glob(os.path.join(IMG_DIR, img_name + '*'))[0]
            im = Image.open(p).convert('RGB').resize((IMG_SIZE, IMG_SIZE))
            im.save(os.path.join(FAIL_DIR, f"FN_{cls_name}_{i}_{os.path.basename(p)}"))
        except Exception as e:
            pass
print('Saved failure cases to', FAIL_DIR)

NameError: name 'probs' is not defined

## Done

- Prediction report: /mnt/data/output/val_prediction_report_with_uncertainty.csv
- Grad-CAM overlays: /mnt/data/output/gradcam_overlays
- Failure cases: /mnt/data/output/failure_gallery

Adjust batch size or IMG_DIR if needed.